In [ ]:
from custom_model.models import DGCN, SoftLoss, DERLoss, RegressionLoss
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

import torch
from torch.optim.lr_scheduler import StepLR

from custom_model.utils import *
from custom_model.training import *
from custom_model.custom_dataset import *

from absl import logging
logging._warn_preinit_stderr = 0
logging.warning('Worrying Stuff')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

In [ ]:
para = {}
para['time_invertal'] = 5
para['horizon'] = 15
para['observation'] = 20
para['nb_node'] = 193
para['dim_feature'] = 128
A = adjacency_matrix(3)
B = adjacency_matrixq(3, 8)

years = ['2018', '2019']

dt = np.load('./results/order_train.npz', allow_pickle=True)
order = dt['order']
Nb = len(order)
print(Nb)

In [ ]:
trainset = AMSdataset_distill(years, para, order[int(Nb*0.7):], 'train')
#trainset = AMSdataset_filtered(para, 'high')
print(len(trainset))
validationset = AMSdataset(years, para, 'validation')
validation_loader = DataLoader(validationset, batch_size=16, shuffle=False)
BATCH_SIZE = 16
EPOCH_NUMBER = 80
loss = RegressionLoss().to(device)

In [ ]:
#model = DGCN(para, A, B, uncertainty=False).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
scheduler = StepLR(optimizer, step_size=1, gamma=0.975)
train_model(EPOCH_NUMBER, BATCH_SIZE, trainset, model, optimizer, validation_loader, loss,
                  scheduler, para['horizon'], beta=-0.07)

In [ ]:
torch.save(model.state_dict(), './distill/high70.pt')

In [ ]:
model = DGCN(para, A, B, uncertainty=False).to(device)
model.load_state_dict(torch.load('./distill/high70.pt'))

In [ ]:
para['time_invertal'] = 5
testset = AMSdataset(['2022'], para, 'test')
prediction = test_run_point_distill(testset, model)

In [ ]:
weights = np.where(testset.X[:,-15:,:,0]>0.45, 1., 4.)
np.mean(np.abs(prediction[...,0]-testset.X[:,-15:,:,0])*weights)*130

In [ ]:
np.mean(weights*np.abs(prediction[...,0]-testset.X[:,-15:,:,0])/testset.X[:,-15:,:,0]*100)

In [ ]:
np.mean(weights*(prediction[...,0]-testset.X[:,-15:,:,0])**2)**0.5*130

In [ ]:
percentile = np.array([30, 40, 50, 60, 70, 80, 90, 100])
MAE = np.array([5.97, 6.13, 5.99, 5.89, 5.78, 5.24, 4.88, 4.35])
p1 = np.array([30, 60, 70, 80, 90])
MAE1 = np.array([4.37, 4.41, 4.36, 4.91, 5.36])

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
ax.plot(percentile, MAE, marker='x', label='preserved')
ax.plot(p1, MAE1, marker='o', label='removed')
ax.vlines(52.9, 4.1, 6.9, ls='--', color='black')
#ax.vlines(30.6, 4.1, 6.4, ls='--', color='black')
#ax.axvspan(20, 31.6, alpha=0.5, color='lightblue', label='free-flowing')
ax.axvspan(52.9, 110, alpha=0.3, color='salmon', label='congested')
ax.set_xlabel('percentile (%)')
ax.set_ylabel('weighted MAE (km/h)')
ax.set_ylim(4.1, 6.4)
ax.set_xlim(20, 105)
plt.legend(fontsize=10)
plt.grid()
plt.tight_layout()
plt.savefig('./imgs/distill.pdf', dpi=600)
plt.show()

In [ ]:
dt = np.load('./results/order_2020.npz', allow_pickle=True)
len(dt['remove']) 

In [ ]:
testset = AMSdataset(['2018', '2019'], para, 'train')

In [ ]:
speed = testset.X[order[-14000],...,0]
plt.imshow(speed)
plt.show()

In [ ]:
vmin_out = np.amin(testset.X[:,-15:,...,0], (1,2))
vmin_in = np.amin(testset.X[:,:20,...,0], (1,2))

In [ ]:
len(np.where((vmin_in<0.45) & (vmin_out<0.45))[0])

In [ ]:
1-16099/Nb

In [ ]:
plt.imshow(prediction[order[-18000],...,0],vmin=0, vmax=1)
plt.show()

In [ ]:
130*0.4